In [1]:
import json
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import DataLoader, Dataset

In [2]:
class CoverLetterDataset(Dataset):
    def __init__(self, file_path):
        self.data = self.load_data(file_path)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

        if '[PAD]' not in self.tokenizer.get_vocab():
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data[idx]
        prompt = example['prompt']
        cover_letter = example['cover_letter']

        # Tokenize the prompt and cover letter
        inputs = self.tokenizer.encode_plus(
            prompt,
            cover_letter,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=256,
            return_tensors='pt'
        )
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze()
        }
    
    def load_data(self, file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        return data

In [24]:
# Define the model architecture
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define your training parameters
batch_size = 16
num_epochs = 20
learning_rate = 2e-5

# Create a DataLoader for batching and shuffling the data
dataset = CoverLetterDataset('../data/textgen.json')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Set device (GPU if available, else CPU)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Set the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")

# Save the trained model
model.save_pretrained('../models/textgen')


Epoch 1/20 - Average Loss: 2.0109
Epoch 2/20 - Average Loss: 1.7569
Epoch 3/20 - Average Loss: 1.6232
Epoch 4/20 - Average Loss: 1.4416
Epoch 5/20 - Average Loss: 1.3798
Epoch 6/20 - Average Loss: 1.3219
Epoch 7/20 - Average Loss: 1.2337
Epoch 8/20 - Average Loss: 1.1784
Epoch 9/20 - Average Loss: 1.0761
Epoch 10/20 - Average Loss: 1.0751
Epoch 11/20 - Average Loss: 0.9845
Epoch 12/20 - Average Loss: 1.0161
Epoch 13/20 - Average Loss: 0.9059
Epoch 14/20 - Average Loss: 0.8684
Epoch 15/20 - Average Loss: 0.8913
Epoch 16/20 - Average Loss: 0.7837
Epoch 17/20 - Average Loss: 0.8256
Epoch 18/20 - Average Loss: 0.7305
Epoch 19/20 - Average Loss: 0.7495
Epoch 20/20 - Average Loss: 0.6566


In [25]:

# Load the fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('../models/textgen')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

if '[PAD]' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Set the model to evaluation mode

# Define the prompts for cover letter generation
prompts = [
    "generate a cover letter for software engineering",
    "write cover letter for marketing",
    "write me a cover letter",
    "write a cover letter for data science position"
]

# Generate cover letters
for prompt in prompts:
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # Generate the cover letter text
    generated_ids = model.generate(input_ids=input_ids, max_length=200, num_return_sequences=1)
    
    # Decode the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    # Print the generated cover letter
    print("Prompt:", prompt)
    print("Generated Cover Letter:")
    print(generated_text)
    print("-------------------------------\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: generate a cover letter for software engineering
Generated Cover Letter:
generate a cover letter for software engineering position

Dear Hiring Manager,

I am writing to apply for the software engineering position at your company. With a strong background in software engineering and a passion for delivering exceptional software solutions, I am confident in my ability to contribute to your team.

In my previous role at Microsoft, I successfully designed and delivered software solutions for Microsoft Office 365, Office 365 Business, Office 365 Education, Office 365 Business Insider, Office 365 Business Insider Professional, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Office 365 Business Insider Enterprise, Of

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: write cover letter for marketing
Generated Cover Letter:
write cover letter for marketing manager

Dear Hiring Manager,

I am writing to apply for the marketing manager position at your company. With a strong background in marketing and a passion for delivering compelling and engaging marketing messages, I am confident in my ability to contribute to your team.

In my previous role at XYZ Company, I successfully targeted and targeted targeted audiences for marketing campaigns, including television, radio, and print media. I am skilled in using digital technologies to deliver compelling marketing messages, including mobile apps, social media, and mobile analytics.

Furthermore, I possess excellent communication and interpersonal skills. I am skilled in using social media to communicate with customers, colleagues, and colleagues in a timely and effective manner.

I am excited about the opportunity to join your organization and contribute to your marketing efforts. Thank you for co

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: write me a cover letter
Generated Cover Letter:
write me a cover letter for your position at the company.

Dear Hiring Manager,
I am writing to express my interest in the position at your company. With a strong background in business and a passion for delivering exceptional results, I am confident in my ability to contribute to your team.

In my previous role at XYZ Company, I successfully managed XYZ operations and provided critical support to customers and employees. I am skilled in using data and analysis tools to analyze data and identify trends. I am skilled in using data visualization tools to identify trends and identify opportunities for improvement.

Furthermore, I possess excellent communication and interpersonal skills. I am skilled in using social media to communicate with customers and employees. I am skilled in using social media to communicate with stakeholders and stakeholders about important business decisions.

I am excited about the opportunity to join your o